# Exploring the ML Functions of H2O in Python

Here we will explore how to conduct different machine learning functionality using the h2o.ai platform's package for python. 

In [79]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch

In [45]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.
Attempting to start a local H2O server...
  Java Version: java version "17.0.5" 2022-10-18 LTS; Java(TM) SE Runtime Environment (build 17.0.5+9-LTS-191); Java HotSpot(TM) 64-Bit Server VM (build 17.0.5+9-LTS-191, mixed mode, sharing)
  Starting server from /Users/whitespaceuser/opt/anaconda3/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/00/kbrrh3px4tg8q1w03sncfzx80000gn/T/tmpgafuqq2f
  JVM stdout: /var/folders/00/kbrrh3px4tg8q1w03sncfzx80000gn/T/tmpgafuqq2f/h2o_whitespaceuser_started_from_python.out
  JVM stderr: /var/folders/00/kbrrh3px4tg8q1w03sncfzx80000gn/T/tmpgafuqq2f/h2o_whitespaceuser_started_from_python.err
  Server is running at http://127.0.0.1:54325
Connecting to H2O server at http://127.0.0.1:54325 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.2
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_whitespaceuser_6y5r21
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


### Data Loading and Preprocessing

The data comes to us today from the following kaggle source: https://www.kaggle.com/datasets/alopez247/pokemon

In [46]:
df = h2o.import_file("pokemon_alopez247.csv")
df.head()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,isLegendary,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,Green,True,0.875,Monster,Grass,False,0.71,6.9,45,quadruped
2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,Green,True,0.875,Monster,Grass,False,0.99,13,45,quadruped
3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,Green,True,0.875,Monster,Grass,True,2.01,100,45,quadruped
4,Charmander,Fire,,309,39,52,43,60,50,65,1,False,Red,True,0.875,Monster,Dragon,False,0.61,8.5,45,bipedal_tailed
5,Charmeleon,Fire,,405,58,64,58,80,65,80,1,False,Red,True,0.875,Monster,Dragon,False,1.09,19,45,bipedal_tailed
6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,Red,True,0.875,Monster,Dragon,True,1.7,90.5,45,bipedal_tailed
7,Squirtle,Water,,314,44,48,65,50,64,43,1,False,Blue,True,0.875,Monster,Water_1,False,0.51,9,45,bipedal_tailed
8,Wartortle,Water,,405,59,63,80,65,80,58,1,False,Blue,True,0.875,Monster,Water_1,False,0.99,22.5,45,bipedal_tailed
9,Blastoise,Water,,530,79,83,100,85,105,78,1,False,Blue,True,0.875,Monster,Water_1,True,1.6,85.5,45,bipedal_tailed
10,Caterpie,Bug,,195,45,30,35,20,20,45,1,False,Green,True,0.5,Bug,,False,0.3,2.9,255,insectoid


In [47]:
df.describe()

Rows:721
Cols:23

,Number,Name,Type_1,Type_2,Total,HP,Attack,Defense,Sp_Atk,Sp_Def,Speed,Generation,isLegendary,Color,hasGender,Pr_Male,Egg_Group_1,Egg_Group_2,hasMegaEvolution,Height_m,Weight_kg,Catch_Rate,Body_Style
type,int,string,enum,enum,int,int,int,int,int,int,int,int,enum,enum,enum,real,enum,enum,enum,real,real,int,enum
mins,1.0,NaN,,,180.0,1.0,5.0,5.0,10.0,20.0,5.0,1.0,,,,0.0,,,,0.1,0.1,3.0,
mean,361.0,NaN,,,417.94590846047157,68.38002773925103,75.01386962552012,70.80859916782246,68.7378640776699,69.29126213592232,65.71428571428571,3.323162274618585,,,,0.5533773291925466,,,,1.1449791955617197,56.77337031900139,100.24687933425797,
maxs,721.0,NaN,,,720.0,255.0,165.0,230.0,154.0,230.0,160.0,6.0,,,,1.0,,,,14.5,950.0,255.0,
sigma,208.2790595971344,NaN,,,109.66367074447545,25.848271820575174,28.984475281886887,29.296558071563616,28.7880052257176,27.01586043808999,27.27792002031901,1.669873244529968,,,,0.19996900735371828,,,,1.0443685124726578,89.09566681624158,76.57351274971818,
zeros,0,0,,,0,0,0,0,0,0,0,0,,,,23,,,,0,0,0,
missing,0,0,0,371,0,0,0,0,0,0,0,0,0,0,0,77,0,530,0,0,0,0,0
0,1.0,Bulbasaur,Grass,Poison,318.0,45.0,49.0,49.0,65.0,65.0,45.0,1.0,False,Green,True,0.875,Monster,Grass,False,0.71,6.9,45.0,quadruped
1,2.0,Ivysaur,Grass,Poison,405.0,60.0,62.0,63.0,80.0,80.0,60.0,1.0,False,Green,True,0.875,Monster,Grass,False,0.99,13.0,45.0,quadruped
2,3.0,Venusaur,Grass,Poison,525.0,80.0,82.0,83.0,100.0,100.0,80.0,1.0,False,Green,True,0.875,Monster,Grass,True,2.01,100.0,45.0,quadruped


In [48]:
df["hasMegaEvolution"] = df["hasMegaEvolution"].asfactor()

In [59]:
df["hasMegaEvolution"].table()

hasMegaEvolution,Count
False,675
True,46


In [49]:
splits = df.split_frame(ratios=[0.7, 0.15], seed=123)  

train = splits[0]
valid = splits[1]
test = splits[2]

In [50]:
#Our response will be if the pokemon has a "mega evolution"
y = "hasMegaEvolution"
x = list(df.columns)
x.remove("Number")
x.remove("Name")
x.remove("hasMegaEvolution")

## Machine Learning

Here we will try to use a few tools from H2O Ai to build models. Since our response variable is true or false, this is just binary classification.

### Building a Single Logistic Regression using a GLM

In [51]:
glm_fit = H2OGeneralizedLinearEstimator(family='binomial', 
                                        model_id='GLM_model')

In [53]:
glm_model = glm_fit.train(x=x, y=y, training_frame=train)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


We will instantly see output from the model when we call for it. This includes the model parameters, details of the training, variable importance, confusion matrices, and metrics of the model on the training set.

In [54]:
print(glm_model)

Model Details
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_model


GLM Model: summary
    family    link    regularization                                number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  --------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   Elastic Net (alpha = 0.5, lambda = 0.01101 )  104                           14                             6                       py_16_sid_b790

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.051499301680716654
RMSE: 0.22693457577177756
LogLoss: 0.18896489735967728
AUC: 0.8378754171301446
AUCPR: 0.3036260661968141
Gini: 0.6757508342602891
Null degrees of freedom: 494
Residual degrees of freedom: 480
Null deviance: 231.79211379895935
Residual deviance: 187.07524838608052
AIC: 217.07524838608052

Co


Next we will assess the model's performance on the test data

In [56]:
glm_performance = glm_model.model_performance(test)

In [57]:
glm_performance

ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.0512002597547088
RMSE: 0.22627474396120484
LogLoss: 0.17652319865295496
AUC: 0.9314903846153846
AUCPR: 0.5593671592774458
Gini: 0.8629807692307692
Null degrees of freedom: 111
Residual degrees of freedom: 97
Null deviance: 57.78115668056316
Residual deviance: 39.54119649826191
AIC: 69.54119649826191

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.1426828532504471
       False    True    Error    Rate
-----  -------  ------  -------  -----------
False  99       5       0.0481   (5.0/104.0)
True   3        5       0.375    (3.0/8.0)
Total  102      10      0.0714   (8.0/112.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.142683     0.555556  9
max f2                       0.11083      0.686275  18
max f0point5                 0.370066     0.625     1
max accuracy                 0.370066     0.946429  1
max precision                0.422852     1         0
max recall                   0.0752607    1         33
max specificity              0.422852     1         0
max absolute_mcc             0.131879     0.524142  13
max min_per_class_accuracy   0.11083      0.875     18
max mean_per_class_accuracy  0.11083      0.879808  18
max tns                      0.422852     104       0
max fns                      0.422852     7         0
max fps                      0.00357626   104       111
max tps                      0.0752607    8         33
max tnr                      0.422852     1         0
max fnr                      0.422852     0.875     0
max fpr                      0.00357626   1         111
max tpr                      0.0752607    1         33

Gains/Lift Table: Avg response rate:  7.14 %, avg score:  6.27 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0178571                   0.35854            14       14                 1                0.396459    1                           0.396459            0.25            0.25                       1300     1300               0.25
2        0.0267857                   0.256261           0        9.33333            0                0.265283    0.666667                    0.352734            0               0.25                       -100     833.333            0.240385
3        0.0357143                   0.219049           0        7                  0                0.224272    0.5                         0.320618            0               0.25                       -100     600                0.230769
4        0.0446429                   0.208389           14       8.4                1                0.208444    0.6                         0.298183            0.125           0.375                      1300     740                0.355769
5        0.0535714                   0.190435           0        7                  0                0.20832     0.5                         0.283206            0               0.375                      -100     600                0.346154
6        0.107143                    0.134201           4.66667  5.83333            0.333333         0.148117    0.416667                    0.215662            0.25            0.625                      366.667  483.333            0.557692
7        0.151786                    0.117121           2.8      4.94118            0.2              0.129174    0.352941                    0.190224            0.125    

We can access individual aspects of the results by accessing them in the performance variable. 

In [63]:
#The test auc
glm_performance.auc()

0.9314903846153846

We can also see the predictions themselves

In [61]:
preds = glm_model.predict(test)

glm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [64]:
preds.head()

predict,False,True
True,0.847871,0.152129
False,0.989928,0.0100724
False,0.924739,0.0752607
False,0.924824,0.0751761
False,0.917645,0.082355
False,0.903614,0.096386
False,0.949582,0.0504183
False,0.9794,0.0206004
False,0.906853,0.0931471
False,0.916282,0.0837176


Here we see that the model reset a lower threshold for predicting a positive class on its own to get the best GLM (hence why a 0.152 was predicted as true). This is a pretty powerful feature!

Also we can see the scoring history of a model with its "scoring_history" method. This is another cool feature to track training with.

In [66]:
glm_model.scoring_history()

,,timestamp,duration,iterations,negative_log_likelihood,objective,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2022-11-08 13:14:17,0.000 sec,0,115.896057,0.234133,,,,,,,
1,,2022-11-08 13:14:17,0.004 sec,1,101.693262,0.217374,,,,,,,
2,,2022-11-08 13:14:18,0.006 sec,2,97.333881,0.207909,,,,,,,
3,,2022-11-08 13:14:18,0.007 sec,3,96.894198,0.207556,,,,,,,
4,,2022-11-08 13:14:18,0.009 sec,4,96.883676,0.207553,,,,,,,
5,,2022-11-08 13:14:18,0.013 sec,5,93.500330,0.206236,,,,,,,
6,,2022-11-08 13:14:18,0.015 sec,6,93.537624,0.206229,0.226935,0.188965,0.122732,0.837875,0.303626,6.387097,0.10101


### Implementing H2O's Version of a Deep NN

Here is a version of a neural network architecture from H2O, they are pretty customizable with just a few parameters. We won't get too fancy here.

In [75]:
deep_nn = H2ODeepLearningEstimator(epochs=25, stopping_metric='AUC', stopping_tolerance=0.0005, seed=123)
deep_nn = deep_nn.train(x=x, y=y, training_frame=train, validation_frame=valid)

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%



All of its attributes should be the same as GLM.

In [76]:
deep_nn_performance = deep_nn.model_performance(test)

In [77]:
print(deep_nn_performance)

ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.06616545086677415
RMSE: 0.2572264583334579
LogLoss: 0.4362432583634951
Mean Per-Class Error: 0.25961538461538464
AUC: 0.8966346153846154
AUCPR: 0.559696819566174
Gini: 0.7932692307692308

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.014405727303935545
       False    True    Error    Rate
-----  -------  ------  -------  -----------
False  102      2       0.0192   (2.0/104.0)
True   4        4       0.5      (4.0/8.0)
Total  106      6       0.0536   (6.0/112.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.0144057    0.571429  5
max f2                       0.00212928   0.666667  12
max f0point5                 0.116339     0.625     1
max accuracy                 0.116339     0.946429  1
max precision                0.159091     1         0


### Taking a bit more control: Using Grid Search

We can use a bit more manual and slightly more expansive search in H2O by using a manual grid search. 

In [85]:
gradient_boost_params = {'learn_rate': [0.01, 0.1], 
                'max_depth': [5, 10, 20],
                'sample_rate': [0.8, 1.0],
                'col_sample_rate': [0.5, 1.0]}

In [86]:
gbm_grid_model = H2OGridSearch(model=H2OGradientBoostingEstimator,
                          grid_id='gbm_grid_search',
                          hyper_params= gradient_boost_params)
gbm_grid_model = gbm_grid_model.train(x=x, y=y, 
                training_frame=train, 
                validation_frame=valid, 
                ntrees=100,
                seed=123)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


We can then access the complete table of results!

In [88]:
gbm_grid_results = gbm_grid_model.get_grid(sort_by='auc', decreasing=True)
print(gbm_grid_results)

Hyper-Parameter Search Summary: ordered by decreasing auc
     col_sample_rate    learn_rate    max_depth    sample_rate    model_ids                 auc
---  -----------------  ------------  -----------  -------------  ------------------------  ------------------
     0.2                0.1           5.0          0.8            gbm_grid_search_model_7   0.9332443257676903
     1.0                0.1           5.0          0.8            gbm_grid_search_model_9   0.923898531375167
     0.2                0.1           5.0          1.0            gbm_grid_search_model_34  0.9198931909212283
     0.2                0.1           20.0         0.8            gbm_grid_search_model_25  0.9038718291054739
     0.5                0.1           5.0          1.0            gbm_grid_search_model_35  0.9038718291054739
     0.5                0.1           20.0         1.0            gbm_grid_search_model_67  0.9038718291054739
     0.5                0.01          5.0          1.0            gbm_

After doing grid search, we can derive the best model by the scoring metric we set (AUC here).

In [89]:
best_gbm_model = gbm_grid_results.models[0]

In [90]:
gbm_performance = best_gbm_model.model_performance(test)
print(gbm_performance.auc())

0.890625


### Using their AutoML to test many different models!

Next let's use one of H2O's flagship tools, AutoML, to test how it goes about itteratively training models. This will itteratively search through a bunch of models of different types and it cross validates them under different parameter settings. This is a great way to automate conducting ML training!

In [98]:
auto_ml = H2OAutoML(max_models=100, seed=123)
auto_ml.train(x=x, y=y, training_frame=train, validation_frame=valid)

AutoML progress: |
15:16:21.494: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.

███████████████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_5_20221108_151621_model_31


Model Summary: 
    number_of_trees
--  -----------------
    40

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.004397107884099591
RMSE: 0.06631069207978145
LogLoss: 0.031348321301000674
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6015811562538147
       False    True    Error    Rate
-----  -------  ------  -------  -----------
False  464      0       0        (0.0/464.0)
True   0        31      0        (0.0/31.0)
Total  464      31      0        (0.0/495.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.601581     1         30
max f2                       0.601581     1         30
max f0point5                 0.601581     1         30
max accuracy                 0.601581     1         30
max precision                0.920541     1         0
max recall                   0.601581     1         30
max specificity              0.920541     1         0
max absolute_mcc             0.601581     1         30
max min_per_class_accuracy   0.601581     1         30
max mean_per_class_accuracy  0.601581     1         30
max tns                      0.920541     464       0
max fns                      0.920541     30        0
max fps                      0.000724016  464       399
max tps                      0.601581     31        30
max tnr                      0.920541     1         0
max fnr                      0.920541     0.967742  0
max fpr                      0.000724016  1         399
max tpr                      0.601581     1         30

Gains/Lift Table: Avg response rate:  6.26 %, avg score:  6.37 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.010101                    0.8858             15.9677  15.9677            1                0.905194    1                           0.905194            0.16129         0.16129                    1496.77  1496.77            0.16129
2        0.020202                    0.804451           15.9677  15.9677            1                0.857033    1                           0.881114            0.16129         0.322581                   1496.77  1496.77            0.322581
3        0.030303                    0.780825           15.9677  15.9677            1                0.790652    1                           0.85096             0.16129         0.483871                   1496.77  1496.77            0.483871
4        0.040404                    0.741544           15.9677  15.9677            1                0.766087    1                           0.829742            0.16129         0.645161                   1496.77  1496.77            0.645161
5        0.0505051                   0.717475           15.9677  15.9677            1                0.732515    1                           0.810296            0.16129         0.806452                   1496.77  1496.77            0.806452
6        0.10101                     0.0800504          3.83226  9.9                0.24             0.245943    0.62                        0.52812             0.193548        1                          283.226  890                0.959052
7        0.151515                    0.0419423          0        6.6                0                0.0575887   0.413333                    0.37

We need to access the leaderboard of the auto ML, and from this can get us the best models.

In [ ]:
leaderboard = auto_ml.leaderboard

In [ ]:
leaderboard.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_grid_1_AutoML_5_20221108_151621_model_31,0.852162,0.200224,0.232237,0.231368,0.234933,0.0551935
XGBoost_grid_1_AutoML_5_20221108_151621_model_4,0.848686,0.189064,0.26575,0.296858,0.230379,0.0530746
XGBoost_grid_1_AutoML_5_20221108_151621_model_15,0.840865,0.191249,0.238737,0.21736,0.23219,0.0539122
XGBoost_grid_1_AutoML_5_20221108_151621_model_17,0.835825,0.191453,0.229675,0.236721,0.230946,0.0533361
XGBoost_grid_1_AutoML_5_20221108_151621_model_5,0.832175,0.199065,0.212534,0.288306,0.236145,0.0557647
StackedEnsemble_BestOfFamily_1_AutoML_5_20221108_151621,0.830749,0.191957,0.218885,0.299013,0.23122,0.0534629
XGBoost_grid_1_AutoML_5_20221108_151621_model_24,0.82988,0.232962,0.207107,0.186318,0.242249,0.0586844
XGBoost_grid_1_AutoML_5_20221108_151621_model_13,0.829116,0.192604,0.231238,0.286117,0.231568,0.0536238
XGBoost_grid_1_AutoML_5_20221108_151621_model_27,0.82776,0.200995,0.240976,0.256083,0.236072,0.0557298
XGBoost_grid_1_AutoML_5_20221108_151621_model_12,0.825153,0.197006,0.227471,0.270022,0.233256,0.0544082


Furthermore, gettting just the "leader" gets the best model

In [ ]:
auto_ml.leader

Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_grid_1_AutoML_5_20221108_151621_model_31


Model Summary: 
    number_of_trees
--  -----------------
    40

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.004397107884099591
RMSE: 0.06631069207978145
LogLoss: 0.031348321301000674
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6015811562538147
       False    True    Error    Rate
-----  -------  ------  -------  -----------
False  464      0       0        (0.0/464.0)
True   0        31      0        (0.0/31.0)
Total  464      31      0        (0.0/495.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.601581     1         30
max f2                       0.601581     1         30
max f0point5                 0.601581     1         30
max accuracy                 0.601581     1         30
max precision                0.920541     1         0
max recall                   0.601581     1         30
max specificity              0.920541     1         0
max absolute_mcc             0.601581     1         30
max min_per_class_accuracy   0.601581     1         30
max mean_per_class_accuracy  0.601581     1         30
max tns                      0.920541     464       0
max fns                      0.920541     30        0
max fps                      0.000724016  464       399
max tps                      0.601581     31        30
max tnr                      0.920541     1         0
max fnr                      0.920541     0.967742  0
max fpr                      0.000724016  1         399
max tpr                      0.601581     1         30

Gains/Lift Table: Avg response rate:  6.26 %, avg score:  6.37 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.010101                    0.8858             15.9677  15.9677            1                0.905194    1                           0.905194            0.16129         0.16129                    1496.77  1496.77            0.16129
2        0.020202                    0.804451           15.9677  15.9677            1                0.857033    1                           0.881114            0.16129         0.322581                   1496.77  1496.77            0.322581
3        0.030303                    0.780825           15.9677  15.9677            1                0.790652    1                           0.85096             0.16129         0.483871                   1496.77  1496.77            0.483871
4        0.040404                    0.741544           15.9677  15.9677            1                0.766087    1                           0.829742            0.16129         0.645161                   1496.77  1496.77            0.645161
5        0.0505051                   0.717475           15.9677  15.9677            1                0.732515    1                           0.810296            0.16129         0.806452                   1496.77  1496.77            0.806452
6        0.10101                     0.0800504          3.83226  9.9                0.24             0.245943    0.62                        0.52812             0.193548        1                          283.226  890                0.959052
7        0.151515                    0.0419423          0        6.6                0                0.0575887   0.413333                    0.37

Ok! Let's get the performance of the leader model on the test set and then save this!

In [102]:
leader_performance = auto_ml.leader.model_performance(test)

In [103]:
print(leader_performance)

ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.0541986381654056
RMSE: 0.2328060097278539
LogLoss: 0.19368566959812386
Mean Per-Class Error: 0.21153846153846154
AUC: 0.8725961538461539
AUCPR: 0.5076556870314453
Gini: 0.7451923076923077

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.23689664900302887
       False    True    Error    Rate
-----  -------  ------  -------  -----------
False  99       5       0.0481   (5.0/104.0)
True   3        5       0.375    (3.0/8.0)
Total  102      10      0.0714   (8.0/112.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.236897     0.555556  9
max f2                       0.04398      0.636364  22
max f0point5                 0.813256     0.625     1
max accuracy                 0.813256     0.946429  1
max precision                0.892743     1         0
max r

In [ ]:
h2o.save_model(auto_ml.leader, path = ".")